### Implementation of LASER to the corpus
### RU-EN

In [1]:
# Imports
import numpy as np
from scipy import spatial
import pandas as pd
from datetime import datetime

from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

import keras
from keras import models
from keras import layers


Using TensorFlow backend.


Load Dataset

In [2]:
df1 = pd.read_csv("scores.csv")

In [3]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17980 entries, 0 to 17979
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   source       17980 non-null  object 
 1   reference    17980 non-null  object 
 2   translation  17980 non-null  object 
 3   z-score      17980 non-null  float64
 4   avg-score    17980 non-null  float64
 5   annotators   17980 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 842.9+ KB


In [4]:
df1.head()

,source,reference,translation,z-score,avg-score,annotators
0,В этом году крымчане получат уведомление на оп...,This year the Crimeans will receive a notice f...,"This year, residents of Crimea will receive a ...",0.878043,92.0,1
1,Энергетические компании находятся под давление...,Energy companies are under pressure due to low...,Energy companies are under pressure from lower...,0.511473,81.5,2
2,В бархатный сезон покупают туры в основном оди...,"In the velvet season, tours are mostly single ...","In the autumn season, tours are mainly purchas...",0.947866,94.0,1
3,Возле него на всякий случай стоит познавательн...,"Near him, just in case, there is a cognitive t...","Beside it, for good measure, there is an infor...",1.052601,97.0,1
4,Действительно ли Эфиопия находится на грани ра...,Is Ethiopia on the verge of schism?,Is Ethiopia about to crack?,0.738397,88.0,1


Comparison of LASER embeddings

In [5]:
source_arr = np.load("laser.source_embeds.npy")
refer_arr = np.load("laser.reference_embeds.npy")
trans_arr = np.load("laser.translation_embeds.npy")

Cosine Similarity

In [6]:
cos_similarity_ref_hyp = []
cos_similarity_src_ref = []
cos_similarity_src_hyp = []

# for each of the cos similarity, put them into lists.
for i in range(refer_arr.shape[0]):
    cos_similarity_ref_hyp.append((spatial.distance.cosine(refer_arr[i],trans_arr[i])*-1)+1)
    cos_similarity_src_ref.append((spatial.distance.cosine(source_arr[i],refer_arr[i])*-1)+1)
    cos_similarity_src_hyp.append((spatial.distance.cosine(source_arr[i],trans_arr[i])*-1)+1)


# Standardize and transform into series.
scaler = StandardScaler()
cs_rh = pd.Series(scaler.fit_transform(np.array(cos_similarity_ref_hyp).reshape(-1,1)).flatten(),name="cos_sim_ref_hyp")
cs_sr = pd.Series(scaler.fit_transform(np.array(cos_similarity_src_ref).reshape(-1,1)).flatten(),name="cos_sim_src_ref")
cs_sh = pd.Series(scaler.fit_transform(np.array(cos_similarity_src_hyp).reshape(-1,1)).flatten(),name="cos_sim_src_hyp")



In [7]:
cos_df = pd.concat([cs_rh,cs_sr,cs_sh,df1.loc[:,"z-score"]],axis=1)
cos_df.head()

,cos_sim_ref_hyp,cos_sim_src_ref,cos_sim_src_hyp,z-score
0,0.090981,0.645514,0.332329,0.878043
1,-0.196355,0.525212,0.096179,0.511473
2,-0.341700,-0.945737,-0.372678,0.947866
3,-0.203153,-0.493851,0.479380,1.052601
4,-2.214009,-1.498225,-2.165453,0.738397


In [8]:
# Pearson
print("Pairwise Pearson")
cos_df.corr()

Pairwise Pearson


,cos_sim_ref_hyp,cos_sim_src_ref,cos_sim_src_hyp,z-score
cos_sim_ref_hyp,1.000000,0.631136,0.741079,0.342903
cos_sim_src_ref,0.631136,1.000000,0.721308,0.195572
cos_sim_src_hyp,0.741079,0.721308,1.000000,0.195184
z-score,0.342903,0.195572,0.195184,1.000000


In [9]:
# Kendall
print("Pairwise Kendall")
cos_df.corr(method="kendall")

Pairwise Kendall


,cos_sim_ref_hyp,cos_sim_src_ref,cos_sim_src_hyp,z-score
cos_sim_ref_hyp,1.000000,0.399261,0.544460,0.236634
cos_sim_src_ref,0.399261,1.000000,0.546191,0.116234
cos_sim_src_hyp,0.544460,0.546191,1.000000,0.118514
z-score,0.236634,0.116234,0.118514,1.000000


In [10]:
# Mean Absolute Deviation of each of these.
for col in cos_df.columns[:3]:
    print(f"Mean Absolute Error of {col} in regards to the z-score: {mean_absolute_error(cos_df[col],cos_df['z-score'])}")

Mean Absolute Error of cos_sim_ref_hyp in regards to the z-score: 0.7842597127484974
Mean Absolute Error of cos_sim_src_ref in regards to the z-score: 0.8609030276643188
Mean Absolute Error of cos_sim_src_hyp in regards to the z-score: 0.875004989889389


---
Linear Regression <br>to try and predict avg score based on the cos similarity 

In [11]:
X = cos_df.drop(columns='z-score')
y = cos_df['z-score']

In [12]:
lin_model = LinearRegression()

In [13]:
kf = KFold(n_splits = 10)
for train_index, test_index in kf.split(X):
    X_train, X_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    
    lin_model.fit(X_train,y_train)
    y_pred = lin_model.predict(X_val)
    lin_model_3_mae = mean_absolute_error(y_val,y_pred)
    print(lin_model_3_mae)

0.5960710557562018
0.6174259543915787
0.6512767939130871
0.6551620429389416
0.6574396729630018
0.6376470406192327
0.6008900363054817
0.6198715246756239
0.712151312874924
0.6378996643778573


In [14]:
lin_model_3_par = [lin_model.intercept_]
lin_model_3_par.extend([coef for coef in lin_model.coef_])
print(lin_model_3_par)

[0.0034786935062026455, 0.38461416565797896, 0.03342224180059689, -0.13847138238775347]


In [15]:
# A model with src_ref and src_hyp only. 
X = cos_df.drop(columns=['cos_sim_ref_hyp','z-score'])
y = cos_df['z-score']

In [16]:
kf = KFold(n_splits = 10)
for train_index, test_index in kf.split(X):
    X_train, X_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    
    lin_model.fit(X_train,y_train)
    y_pred = lin_model.predict(X_val)
    print(mean_absolute_error(y_val,y_pred))

0.6667819975210332
0.6538828380941704
0.6946735070511983
0.6983276024198847
0.6920425300385376
0.6735663057159539
0.6448266327638793
0.6639671106266699
0.7351340916537735
0.6734323696134786


In [17]:
y_pred

array([ 0.05456465, -0.04641414,  0.12301479, ..., -0.34584902,
       -0.16493327,  0.00259433])

In [18]:
lin_model_2_par = [lin_model.intercept_]
lin_model_2_par.extend([coef for coef in lin_model.coef_])
print(lin_model_2_par)

[0.003093191432067345, 0.1122625940476956, 0.09158706022470825]


---
Using a Neural Network

In [19]:
MLP_model = MLPRegressor(hidden_layer_sizes=(2,2),max_iter=250,learning_rate_init=0.0015,solver="lbfgs")

In [20]:
# Train test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [21]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [22]:
kf = KFold(5)
for train_index, val_index in kf.split(X_train):
    kf_X_train, X_val = X_train[train_index], X_train[val_index]
    kf_y_train, y_val = y_train.iloc[train_index], y_train.iloc[val_index]    
    
    MLP_model.fit(kf_X_train,kf_y_train)
    y_val_pred = MLP_model.predict(X_val)
    print(f"Model loss: {MLP_model.loss_}")
    print(f"The MAE is {mean_absolute_error(y_val,y_val_pred)}")

Model loss: 0.3602669013436487
The MAE is 0.7080631343328468
Model loss: 0.3702645546192668
The MAE is 0.6700767657691566
Model loss: 0.37060335845093473
The MAE is 0.6641719805499471
Model loss: 0.37784641724397233
The MAE is 0.6655677640072851
Model loss: 0.3670488681820223
The MAE is 0.6738987995005139


---
Using Neural Networks, directly on the embeddings.

In [23]:
# An input has to be of shape (3,1024), because we have 3 embedded vectors of size 1024.
# combined array
#full_arr = np.dstack((source_arr,refer_arr,trans_arr))

# Option 2, make it size (1,2048), by concatenating the arrays. This is what we are using now, according to teacher's indications.
full_arr_c = np.concatenate((refer_arr,trans_arr),axis=1)


In [24]:
X = full_arr_c.copy()
y = df1["z-score"].to_numpy()

# Train test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [25]:
X_train.shape

(16182, 2048)

In [26]:
model = models.Sequential()

model.add(layers.Dense(1024,activation="relu",input_dim=X_train.shape[1]))
model.add(layers.Dense(512,activation="relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128,activation="relu"))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(64,activation="relu"))

model.add(layers.Dense(1,activation="tanh"))
model.compile(optimizer=keras.optimizers.Adam(lr=0.002),loss="mse",metrics=["mae"])

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                

In [ ]:
# Fit. Batch Size < > Learning Rate

model.fit(X_train,y_train,epochs=50,batch_size=64,verbose=0)

In [ ]:
# Evaluation and comparison.
test_mse,test_mae = model.evaluate(X_test,y_test)
test_corr = np.corrcoef(model.predict(X_test).flatten(),y_test)[0][1]
print(f"Test MSE: {test_mse:.4f}.\nTest MAE: {test_mae:.4f}.")
print(f"Pearson correlation between y_val_predicted and actual y_val: {test_corr:.4f}")

In [ ]:
# For reference, pt.2
for col in cos_df.columns[:3]:
    print(f"Mean Absolute Error of {col} in regards to the z-score: {mean_absolute_error(cos_df[col],cos_df['z-score'])}")

In [ ]:
# Save Model.
model.save("ru_en_laser_model__testcorr_{:.4f}_MAE_{:.4f}_time{}.h5".format(test_corr,test_mae,datetime.now().strftime('%d_%m_%H_%M')
))